# Sentence Variational Adversarial Active Learning (S-VAAL)
@author: Tyler Bikaun

The following notebook intends to flesh out an initial proof concept of the coupling of S-VAE (Bowman <i>et al.</i> 2016; https://arxiv.org/abs/1511.06349) and VAAL (Sinha <i>et al.</i> 2019; https://arxiv.org/abs/1904.00370)

<b>Application:</b> Named Entity Task (NER)

<b>Architecture Diagram:</b>


### Standard Imports

In [351]:
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.nn.utils.rnn as rnn_utils
import torch.optim as optim

torch.manual_seed(1)

## Testing Functions
These functions are used for spot testing code whilst developing. For example, building random sequences of tensors.
- [x] Build artificial sequence generator

In [498]:
class Tester:
    def __init__(self):
        pass
    
    def build_sequences(self, batch_size: int, max_seq_len: int) -> int:
        """
        Builds tensor of specified size containing variable length, padded, sequences of integers
            
        Arguments
        ---------
            batch_size : int
                Number of sequences to generate
            max_seq_len : int
                Maximum length of sequences
        Returns
        -------
            sequences : tensor
                Tensor of generated sequences
            lengths : tensor
                Tensor of sequence lengths
        """
        seqs = list()
        for i in range(batch_size):
            # Generate random integer sequences
            seq = np.random.randint(low=1, high=100, size=(random.randint(1, max_seq_len),))
            # Add padding
            seq = np.concatenate((seq, np.zeros(shape=(max_seq_len - len(seq)))), axis=None)
            seqs.append(seq)
        sequences = torch.LongTensor(seqs)
        lengths = torch.tensor([len(seq[seq != 0]) for seq in sequences])
        
        print(f'Shapes - seq {sequences.shape} - lengths {lengths.shape}')
        
        return sequences, lengths
    
    def build_sequence_tags(self, sequences, label_space_size: int):
        """
        Given a set of sequences, generates ground truth labels
        
        Arguments
        ---------
            sequences : tensor
                Tensor of generated sequences
            label_space_size : int
                Size of label space
        Returns
        -------
            X,y : list of tuples
                Artificial ground truth dataset
                    X dim : (seq batch size)
                    y dim : (seq batch size, label space size)
        """
        
        batch_size = len(sequences)
        
        dataset = list()
        for sequence in sequences:
            # each 'token' in the sequence has a label mapping
            label_set = list()
            for token in sequence:
                one_hot_array = np.zeros(shape=(label_space_size))
                one_hot_array[random.randint(0,label_space_size-1)]=1
                labels = torch.IntTensor(one_hot_array)
                label_set.append(labels)
            dataset.append((sequence, torch.stack(label_set)))   # stack list of labels into tensors
        return dataset

In [499]:
# Test functionality
tester = Tester()
sequences, lengths = tester.build_sequences(batch_size=10, max_seq_len=10)
dataset = tester.build_sequence_tags(sequences=sequences, label_space_size=4)

Shapes - seq torch.Size([10, 10]) - lengths torch.Size([10])


In [502]:
for X, y in dataset:
    print(f'X {X.shape} - {X}\n')
    print(f'y {y.shape} - {y}')
    break

X torch.Size([10]) - tensor([39,  9, 25, 56, 80,  0,  0,  0,  0,  0])

y torch.Size([10, 4]) - tensor([[1, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0],
        [0, 0, 0, 1]], dtype=torch.int32)


### Utility Functions

- [ ] Build data preprocessor
- [ ] Build data loaders

In [ ]:
# configuration for model building, training, evaluation process. This will be converted into yaml.
config = {'': ''}

### Models
<i>Model architectures</i><br>
<b>SVAE</b> - RNN<br>
<b>Discriminator</b> - FC NN<br>
<b>Task Learner</b> - RNN<br>
- [ ] SVAE
- [ ] Discriminator
- [ ] Task Learner

#### SVAE
To do:
 - [ ] 

In [284]:
class SVAE(nn.Module):
    """ Sentence Variational Autoencoder (Bowman et al. 2016)"""
    
    def __init__(self, vocab_size, embedding_size):
        super(SVAE, self).__init__()
        
        # TODO: fix dodgy vocab_size issue... this will be cleared up when utils implemented properly
        
        self.tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor
        
        self.max_sequence_length = 40    # arg
        self.pad_idx = 0
        self.eos_idx = vocab_size + 1
        self.sos_idx = vocab_size + 2
        self.unk_idx = vocab_size + 3
        
        self.vocab_size = vocab_size + 4
        
        self.z_dim = 8
        
        self.rnn_type = 'gru'
        self.bidirectional = False
        self.num_layers = 1
        self.hidden_size = 128
        
        self.embedding = nn.Embedding(self.vocab_size, embedding_size)
        self.word_dropout_rate = 0.1
        self.embedding_dropout = nn.Dropout(p=0.5)
        
        # set rnn type
        if self.rnn_type == 'gru':
            rnn = nn.GRU
        else:
            raise ValueError()
        
        # init encoder-decoder RNNs (models are identical)
        self.encoder_rnn = rnn(embedding_size,
                               self.hidden_size, 
                               num_layers=self.num_layers,
                               bidirectional=self.bidirectional,
                               batch_first=True)
        self.decoder_rnn = rnn(embedding_size,
                               self.hidden_size, 
                               num_layers=self.num_layers,
                               bidirectional=self.bidirectional,
                               batch_first=True)

        self.hidden_factor = (2 if self.bidirectional else 1) * self.num_layers
        
        # Initialisation of FC layers
        # These go from encoder to latent (z) space
        self.hidden2mean = nn.Linear(self.hidden_size * self.hidden_factor, self.z_dim)
        self.hidden2logv = nn.Linear(self.hidden_size * self.hidden_factor, self.z_dim)
        self.z2hidden = nn.Linear(self.z_dim, self.hidden_size * self.hidden_factor)
        self.outputs2vocab = nn.Linear(self.hidden_size * (2 if self.bidirectional else 1), self.vocab_size)
    
    
    def forward(self, input_sequence, length):
        """ Forward pass through VAE """
        
        batch_size = input_sequence.size(0)
        sorted_lengths, sorted_idx = torch.sort(length, descending=True)   # trick for packed padding
        input_sequence = input_sequence[sorted_idx]
        
        # ENCODER
        input_embedding = self.embedding(input_sequence)
#         print(input_embedding.shape)
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.data.tolist(), batch_first=True)
        _, hidden = self._encode(packed_input)
        
        if self.bidirectional or 1 < self.num_layers:
            # flatten hidden state
            hidden = hidden.view(batch_size, self.hidden_size * self.hidden_factor)
        else:
            # .squeeze() -> Returns a tensor with all the dimensions of input of size 1 removed.
            print(f'hidden shape before squeeze {hidden.shape}')
#             hidden = hidden.squeeze()   # doesn't work? gives wrong dimension down stream...
            pass
            print(f'hidden shape after squeeze {hidden.shape}')

        
        # Reparameterisation trick!
        z, mean, logv, std = self.reparameterise(hidden, batch_size)
        
        # DECODER
        if 0 < self.word_dropout_rate:
            prob = torch.rand(input_sequence.size())

            if torch.cuda.is_available():
                prob = prob.cuda()

            prob[(input_sequence.data - self.sos_idx) * (input_sequence.data - self.pad_idx) == 0] = 1

            decoder_input_sequence = input_sequence.clone()
            
#             print(vocab_size)
#             print(self.unk_idx)
            decoder_input_sequence[prob < self.word_dropout_rate] = self.unk_idx

#             print(decoder_input_sequence)
            input_embedding = self.embedding(decoder_input_sequence)

        input_embedding = self.embedding_dropout(input_embedding)
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.data.tolist(), batch_first=True)
        
        outputs, _ = self._decode(packed_input, hidden)
        
        # process outputs
        # Process outputs
        # Unpack padded sequence
        padded_outputs = rnn_utils.pad_packed_sequence(outputs, batch_first=True)[0]
        padded_outputs = padded_outputs.contiguous()
        _, reversed_idx = torch.sort(sorted_idx)
        padded_outputs = padded_outputs[reversed_idx]
        b, s, _ = padded_outputs.size()

        # Project outputs to vocab
        # e.g. project hidden state into label space...
        logp = nn.functional.log_softmax(self.outputs2vocab(padded_outputs.view(-1, padded_outputs.size(2))), dim=-1)
#         print(f'logp before view {logp.shape}\n')
#         print(f'b {b} s {s} no emb {self.embedding.num_embeddings}')
        logp = logp.view(b, s, self.embedding.num_embeddings)

        # logp - log posterior over label space; mean - tensor Gaussian mean, logv - tensor Gaussian variance, z - VAE latent space 
        return logp, mean, logv, z
    
    def to_var(self, x):
        if torch.cuda.is_available():
            x = x.cuda()
        return x
    
    def reparameterise(self, hidden, batch_size):
        """ Implement reparameterisation trick (Kingma and Welling 2014) """
        
        mean = self.hidden2mean(hidden)
        logv = self.hidden2logv(hidden)
        std = torch.exp(0.5 * logv) 
        
        z = self.to_var(torch.randn([batch_size, self.z_dim]))
        return z * std + mean, mean, logv, std
    
    def _encode(self, x):
        """ x - pack padded sequence """
        return self.encoder_rnn(x)
    
    def _decode(self, x, hidden):
        """ x - pack padded sequence
            hidden - latent tensor"""
        return self.decoder_rnn(x, hidden)

In [285]:
# Testing functionality
vocab_size = 100
hidden_size = 128
SVAE = SVAE(vocab_size, hidden_size)

In [313]:
# Test forward pass of SVAE
svae_ff_seqs, svae_ff_lengths = Tester().build_sequences(batch_size=10, max_seq_len=40)

# Pass sequences and lengths into SVAE forward method
print(SVAE.forward(svae_ff_seqs, svae_ff_lengths))

Shapes - seq torch.Size([10, 40]) - lengths torch.Size([10])
hidden shape before squeeze torch.Size([1, 10, 128])
hidden shape after squeeze torch.Size([1, 10, 128])
(tensor([[[-4.4606, -4.4063, -4.6647,  ..., -4.3373, -4.7585, -4.6186],
         [-4.5217, -4.4854, -4.7991,  ..., -4.3907, -4.6902, -4.5589],
         [-4.4746, -4.7599, -4.8052,  ..., -4.3605, -4.7632, -4.4939],
         ...,
         [-4.5724, -4.5642, -4.5951,  ..., -4.6142, -4.6185, -4.6610],
         [-4.5724, -4.5642, -4.5951,  ..., -4.6142, -4.6185, -4.6610],
         [-4.5724, -4.5642, -4.5951,  ..., -4.6142, -4.6185, -4.6610]],

        [[-4.2812, -4.5231, -4.7630,  ..., -4.6618, -4.7277, -4.4953],
         [-4.5380, -4.6145, -4.7663,  ..., -4.6118, -4.7529, -4.4695],
         [-4.5125, -4.6953, -4.4509,  ..., -4.6962, -4.8859, -4.5477],
         ...,
         [-4.5724, -4.5642, -4.5951,  ..., -4.6142, -4.6185, -4.6610],
         [-4.5724, -4.5642, -4.5951,  ..., -4.6142, -4.6185, -4.6610],
         [-4.5724, -4.

In [321]:
# Reviewing components
print(f'Encoder: {SVAE.encoder_rnn}')
print(f'Decoder: {SVAE.decoder_rnn}')
print(f'hidden2mean: {SVAE.hidden2mean}')
print(f'hidden2logv: {SVAE.hidden2logv}')
print(f'z2hidden: {SVAE.z2hidden}')
print(f'outputs2vocab: {SVAE.outputs2vocab}')

Encoder: GRU(128, 128, batch_first=True)
Decoder: GRU(128, 128, batch_first=True)
hidden2mean: Linear(in_features=128, out_features=8, bias=True)
hidden2logv: Linear(in_features=128, out_features=8, bias=True)
z2hidden: Linear(in_features=8, out_features=128, bias=True)
outputs2vocab: Linear(in_features=128, out_features=104, bias=True)


#### Task Learner
Using PyTorch tutorial implementation (https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html) for rapid development (will implement SoTA in the future)<br><br>
To do:
 - [ ] 

In [488]:
class TaskLearner(nn.Module):
    """ Task learner for NER """
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(TaskLearner, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [489]:
# Testing functionality
tasklearner = TaskLearner(embedding_dim=128, hidden_dim=128, vocab_size=104, tagset_size=4)

In [490]:
# Testing forward functionality
test_seq = torch.randint(0,100,size=(10,))
print(f'Test Sequence shape: {test_seq.shape}')
print(f'Forward output:\n{tasklearner.forward(test_seq)}')

Test Sequence shape: torch.Size([10])
Forward output:
tensor([[-1.3398, -1.3176, -1.3322, -1.5778],
        [-1.3969, -1.3763, -1.3059, -1.4732],
        [-1.3410, -1.3915, -1.3968, -1.4176],
        [-1.4056, -1.3032, -1.4402, -1.4016],
        [-1.4455, -1.3493, -1.3432, -1.4108],
        [-1.3431, -1.3183, -1.3382, -1.5651],
        [-1.3362, -1.3592, -1.3775, -1.4781],
        [-1.3841, -1.3881, -1.3320, -1.4441],
        [-1.2373, -1.5060, -1.3808, -1.4411],
        [-1.2520, -1.3633, -1.3697, -1.5894]], grad_fn=<LogSoftmaxBackward>)


In [491]:
# Testing basic training cycle
seqs, lens = tester.build_sequences(batch_size=10, max_seq_len=10)   # Tester initialised at start of notebook
dataset = tester.build_sequence_tags(sequences=seqs, label_space_size=4)

Shapes - seq torch.Size([10, 10]) - lengths torch.Size([10])


In [492]:
dataset

[(tensor([41, 65, 87, 11, 62, 13, 79, 45, 58,  0]),
  [tensor([0, 0, 1, 0], dtype=torch.int32),
   tensor([1, 0, 0, 0], dtype=torch.int32),
   tensor([1, 0, 0, 0], dtype=torch.int32),
   tensor([0, 0, 0, 1], dtype=torch.int32),
   tensor([0, 0, 0, 1], dtype=torch.int32),
   tensor([0, 1, 0, 0], dtype=torch.int32),
   tensor([0, 0, 0, 1], dtype=torch.int32),
   tensor([0, 0, 1, 0], dtype=torch.int32),
   tensor([0, 0, 0, 1], dtype=torch.int32),
   tensor([0, 1, 0, 0], dtype=torch.int32)]),
 (tensor([87,  9, 90,  2, 43, 36, 19, 14,  0,  0]),
  [tensor([0, 1, 0, 0], dtype=torch.int32),
   tensor([1, 0, 0, 0], dtype=torch.int32),
   tensor([0, 0, 0, 1], dtype=torch.int32),
   tensor([1, 0, 0, 0], dtype=torch.int32),
   tensor([1, 0, 0, 0], dtype=torch.int32),
   tensor([0, 1, 0, 0], dtype=torch.int32),
   tensor([0, 0, 0, 1], dtype=torch.int32),
   tensor([0, 1, 0, 0], dtype=torch.int32),
   tensor([0, 0, 1, 0], dtype=torch.int32),
   tensor([1, 0, 0, 0], dtype=torch.int32)]),
 (tensor([ 3

In [493]:
# Generate vocab off of generated sequences
vocab = list()
for seq in seqs:
      vocab.extend(seq.tolist())
vocab = list(set(vocab))
print(vocab)

[0, 2, 3, 9, 10, 11, 13, 14, 15, 16, 17, 19, 20, 21, 23, 25, 28, 36, 38, 40, 41, 43, 45, 46, 54, 55, 58, 61, 62, 64, 65, 67, 79, 81, 87, 88, 89, 90, 91, 97]


In [494]:
# Test training routine
# vocab_size is max int in vocab ints + 1 as 0 is included...
model = TaskLearner(embedding_dim=128, hidden_dim=128, vocab_size=max(vocab)+1, tagset_size=4)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [495]:
with torch.no_grad():
    inputs = seqs[0]
    # forward pass to get output scores
    tag_scores = model(inputs)
    print(tag_scores)

tensor([[-1.4188, -1.4053, -1.3069, -1.4186],
        [-1.3717, -1.4399, -1.4022, -1.3344],
        [-1.3142, -1.4062, -1.4767, -1.3554],
        [-1.3093, -1.3846, -1.4793, -1.3793],
        [-1.4229, -1.3184, -1.4278, -1.3800],
        [-1.3893, -1.3466, -1.3953, -1.4153],
        [-1.3304, -1.3925, -1.4246, -1.4000],
        [-1.4343, -1.3493, -1.2767, -1.4991],
        [-1.3122, -1.4141, -1.3859, -1.4374],
        [-1.4113, -1.4421, -1.3698, -1.3258]])


In [497]:
for epoch in range(5):
    print(f'Epoch: {epoch}')
    for seq, tags in dataset:
#         print(seq,tags)

        model.zero_grad()
        seq_in = seq
        targets = tags
        
        tag_scores = model(seq_in)
        
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

Epoch: 0


AttributeError: 'list' object has no attribute 'size'

#### Discriminator
To do:
 - [ ] 

In [42]:
class Discriminator(nn.Module):
    """ Adversary architecture for discriminator module """
    
    def __init__(self, z_dim=8):
        super(Discriminator, self).__init__()
        
        self.z_dim = z_dim    # latent space dimension
        
        self.net = nn.Sequential(
                                nn.Linear(z_dim, 128),
                                nn.ReLU(True),
                                nn.Linear(128, 128),
                                nn.ReLU(True),
                                nn.Linear(128,1),
                                nn.Sigmoid()
                                )
        # Exe
        self.weight_init()
        
    def weight_init(self):
        """ Weight initialisation
        
        Using Xavier uniform initialisation rather than Kaiming (I think that is more focused for CV? TODO: investigate)
        See: https://pytorch.org/cppdocs/api/function_namespacetorch_1_1nn_1_1init_1ace282f75916a862c9678343dfd4d5ffe.html
        """
        for block in self._modules:
            for m in self._modules[block]:
                if type(m) == nn.Linear:
                    torch.nn.init.xavier_uniform_(m.weight)
                    m.bias.data.fill_(0.01)
    
    def forward(self, z):
        """ Forward pass through discriminator
        
        Arguments
        --------
            z : tensor
                Tensor derived from SVAE latent space
        """
        return self.net(z)

In [56]:
# Testing functionality
dsc = Discriminator()

# Pass random tensor through forward pass of discriminator
rand_tensor = torch.randn((8,))
print(f'Input: {rand_tensor}')
print(f'Output: {dsc.forward(rand_tensor)}')

Input: tensor([ 0.0312,  1.0590,  0.3862, -0.1136, -1.1375,  1.2425, -1.3496, -0.7329])
Output: tensor([0.5398], grad_fn=<SigmoidBackward>)


## Sampler Routine
Active learning based sample selection for task learner

In [302]:
# Code copied from VAAL and modified for sequence data
class Sampler:
    """ Adversary sampler """
    def __init__(self, budget):
        self.budget = budget
        
    def sample(self, vae, discriminator, data, cuda):
        """ Selective sampling algorithm
        
        Arguments
        ---------
            vae : torch model
                VAE model
            discriminator : torch model
                discriminator model
            data : tensor
                Image data
            cuda : boolean
                GPU flag
        Returns
        -------
            querry_pool_indices: int, list
                List of indices corresponding to sorted (top-K) samples to be sampled from
        """
        all_preds = []
        all_indices = []

        for images, _, indices in data:
            if cuda:
                images = images.cuda()

            with torch.no_grad():
                _, _, mu, _ = vae(images)
                preds = discriminator(mu)

            preds = preds.cpu().data
            all_preds.extend(preds)
            all_indices.extend(indices)

        all_preds = torch.stack(all_preds)
        all_preds = all_preds.view(-1)
        # need to multiply by -1 to be able to use torch.topk 
        all_preds *= -1

        # select the points which the discriminator things are the most likely to be unlabeled
        _, querry_indices = torch.topk(all_preds, int(self.budget))
        querry_pool_indices = np.asarray(all_indices)[querry_indices]

        return querry_pool_indices

In [303]:
# Testing functionality
sampler = Sampler(budget=10)


### Training Routine
<i> Pseudo code</i>
```python
for epoch in max_epochs:
        train(task learner)
            get preds
            calc loss
            zero grads
            backpropigate loss
            update model parameters
        for step in max_steps:
            train(SVAE)
        for step in max_steps:
            train(discriminator)
```<br><br>
To do:
 - [ ] 

In [297]:
class Solver:
    def __init__(self):
        self.epochs = 10
        self.vae_steps = 10
        self.discriminator_steps = 10
        
    def train(self):
        """ Performs model training """
    
        for epoch in range(self.epochs):
            print(f'\nEpoch: {epoch}')
            
            print('Train task learner')
            
            print('Train VAE')
            for step in range(self.vae_steps):
                print(f'VAE Step: {step}')
            
            print('Train Discriminator')
            for step in range(self.discriminator_steps):
                print(f'Discriminator Step: {step}')

In [298]:
slvr = Solver()
slvr.train()


Epoch: 0
Train task learner
Train VAE
VAE Step: 0
VAE Step: 1
VAE Step: 2
VAE Step: 3
VAE Step: 4
VAE Step: 5
VAE Step: 6
VAE Step: 7
VAE Step: 8
VAE Step: 9
Train Discriminator
Discriminator Step: 0
Discriminator Step: 1
Discriminator Step: 2
Discriminator Step: 3
Discriminator Step: 4
Discriminator Step: 5
Discriminator Step: 6
Discriminator Step: 7
Discriminator Step: 8
Discriminator Step: 9

Epoch: 1
Train task learner
Train VAE
VAE Step: 0
VAE Step: 1
VAE Step: 2
VAE Step: 3
VAE Step: 4
VAE Step: 5
VAE Step: 6
VAE Step: 7
VAE Step: 8
VAE Step: 9
Train Discriminator
Discriminator Step: 0
Discriminator Step: 1
Discriminator Step: 2
Discriminator Step: 3
Discriminator Step: 4
Discriminator Step: 5
Discriminator Step: 6
Discriminator Step: 7
Discriminator Step: 8
Discriminator Step: 9

Epoch: 2
Train task learner
Train VAE
VAE Step: 0
VAE Step: 1
VAE Step: 2
VAE Step: 3
VAE Step: 4
VAE Step: 5
VAE Step: 6
VAE Step: 7
VAE Step: 8
VAE Step: 9
Train Discriminator
Discriminator Step: 0
D

### Inference Routine
To do:
 - [ ] 


### Sampling Routine
To do:
 - [ ] 